# BCxP GenAI Hands-on Sessions
## Part 1: OpenAI API Basics

### Finde die passende Pizza

Task: Ein Kunde soll basierend auf der Beschreibung einer Speise eine passende Bestelloption aus den vorgegebenen Speisen (definiert in menu.py) finden können. 

## Setup

In [ ]:
# Install required packages
%pip install openai sentence-transformers matplotlib scikit-learn umap-learn plotly faiss-cpu numpy tabulate pandas

In [ ]:
from openai import OpenAI
import json

from hack_helpers import OPENAI_API_KEY
from menu import MENU

openai_client = OpenAI(api_key = OPENAI_API_KEY)

### Verwendung von OpenAI API **ohne Structured Outputs**:


Führe die nächste Zelle aus und beobachte die Ausgabe. Die Ausgabe ist Textbasiert - wie bei der Nutzung von ChatGPT über die Web- oder die Desktop App. Eine programmatische Verwendung des Outputs ist mit dieser 'unstrukturierten' Ausgabe ungeeignet. 

**Aufgabe:** Führe die nächste Zelle mehrmals aus und verändere den User-Input. Beobachte, wie sich das Format der Ausgabe immer wieder verändert.

**Aufgabe:** Passe den System-Prompt an, um die Struktur der Ausgabe vorzugegeben (Beispielsweise "Gib den Namen genau einer Speise an"). Versuche dann den System-Prompt zu 'hacken' und mit einem beliebigen User-Input eine Ausgabe zu erhalten, die nicht der geforderten Struktur entspricht. Ist dies möglich? 


In [ ]:
# 🥡 Menü
# for item in MENU:
#     print(item)

# 💬 Beispiel-Eingabe
user_input = "Ich möchte gerne etwas mit scharfer Salami oder Pilzen."

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "developer", "content": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."},
        {"role": "user", "content": f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}\n\nBestellung: {user_input}"} # Hier übergeben wir die Speisekarte und die Bestellung" 
    ],
)

# 📤 Ausgabe anzeigen
print(response.output_text)


### Verwendung von OpenAI API **mit Structured Outputs**:

Mit _structured outputs_ kann sichergestellt werden, dass das Format der Ausgabe immer dem selben JSON Schema entspricht. JSON (JavaScript Object Notation) ist ein leichtgewichtiges, textbasiertes Format zur Darstellung strukturierter Daten. Es wird häufig in der Webentwicklung und bei APIs zur Übertragung von Daten zwischen Client und Server verwendet. 

**Aufgabe:** Mache Anpassungen, damit nur eine Speise für die Bestellung empfohlen wird. Welche Möglichkeiten gibt es dies zu realisieren (passe das Prompt oder das Schema an). Welche Vorteile/Nachteile gibt es jeweils bei den unterschiedlichen Ansätzen? Kannst du den 'Prompt-Hacking' Ansatz aus der vorherigen Aufgabe auch unter der Verwendung von _structured outputs_ reproduzieren? 

In [ ]:

import json

# 💬 Beispiel-Eingabe
# user_input = "Ich habe Lust auf eine würzige Pizza mit Salami." # Input mit einer Option
user_input = "Ich hätte gerne etwas mit scharfer Salami oder Peperoni." # Input mit mehreren Optionen

# JSON Schema für Ausgabe: Liste von Speisen, wobei eine Speise aus Id, Name, Beschreibung, Kategorie und Preis bestehen.
menu_items_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "developer", "content": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."},
        {"role": "user", "content": f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}\n\nBestellung: {user_input}"}
    ],
    # Gewünschtes Ausgabeformat wird API-Aufruff übergeben
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": menu_items_schema,
            "strict": True
        }
    }
)
recommendations = json.loads(response.output_text)["items"]

# 📤 Nun können wir über Speiseempfehlung iterieren und diese beispielsweise Schritt für Schritt Anzeigen
for item in recommendations:
    print(item)
